# 1. 라이브러리 설치

In [ ]:
import pandas as pd
import os
import datetime
from dateutil.relativedelta import relativedelta
import warnings
import copy
warnings.filterwarnings(action='ignore')
import plotly.express as px
import plotly.io as pio

# 그래프 한글폰트 해결
from matplotlib import font_manager,rc
font_location="c:/windows/fonts/malgun.ttf"
font_name=font_manager.FontProperties(fname=font_location).get_name()
rc('font',family=font_name)

# 마이너스 부호 깨짐 해결
import matplotlib
matplotlib.rcParams['axes.unicode_minus'] = False

# 2. 사용자 설정

2-1. default 경로 입력

In [ ]:
default_dir = 'C:/19Project/05.TRV/01.Pred_Demand/'
data_dir = default_dir + '0. 데이터/input/'

2-2. 파일 경로, 파일명, 분석 기간 설정

In [ ]:
##### 파일 경로 설정 #####
data_dir_output = default_dir + '0. 데이터/output_20211006_1/'

##### 파일명 설정 #####
calendar_file = '달력 데이터.xlsx'

##### 분석 기간 설정 (분석 마지막 연월 설정) #####
end_ym = '2019-09'

# 3. 데이터 불러오기

3-1. 파일명 설정

In [ ]:
data_file_all = '분석테이블_집계_전체.csv'
data_file_center = '분석테이블_집계_센터차량.csv'
data_file_taxi = '분석테이블_집계_임차택시.csv'
od_file = 'tmp_addr_list.csv'
time_file = 'tmp_time_list.csv'

3-2. 디렉터리 생성

In [ ]:
now = datetime.datetime.now().date().strftime('%Y%m%d')

num = 1

heatmap_dir = default_dir + '3. 승하차 분석_' + now

while os.path.exists(heatmap_dir + '_' + str(num)):
    num += 1

heatmap_dir_num = heatmap_dir + '_' + str(num) + '/'

os.makedirs(os.path.join(heatmap_dir_num))

3-3. 집계 데이터, 승하차 구역 리스트 불러오기

In [ ]:
data_all = pd.read_csv(data_dir_output + data_file_all, encoding='cp949', engine = 'python', parse_dates=[0])
data_center = pd.read_csv(data_dir_output + data_file_center, encoding='cp949', engine = 'python', parse_dates=[0])
data_taxi = pd.read_csv(data_dir_output + data_file_taxi, encoding='cp949', engine = 'python', parse_dates=[0])

calendar_data = pd.read_excel(data_dir + calendar_file, converters = {0: pd.to_datetime, 1: int})
od_data = pd.read_csv(data_dir_output + od_file, encoding='cp949', engine = 'python')
time_data = pd.read_csv(data_dir_output + time_file, encoding='cp949', engine = 'python')

3-4. 달력데이터 수정

In [ ]:
# 일자 특성 구분 함수
def f_calendar(x):
    if x in [1, 4, 6] : return '평일'
    elif x in [2, 3, 5, 7]: return '휴일'

calendar_data['일자 특성 구분'] = calendar_data['일자 특성'].apply(lambda x: f_calendar(x))
del calendar_data['일자 특성']
calendar_data.columns = ['승차 일자', '승차 일자 특성 구분']

# 4. 함수 생성

In [ ]:
# 분석 테이블 생성 함수 -> heatmap 함수 내에서 사용
def create_table_heatmap(data, start, end):
    # 기간 설정
    start = pd.to_datetime(start)
    end = pd.to_datetime(end)
    df = data[(data['승차 일자'] >= start) & (data['승차 일자'] <= end)]    
    # 승하차 구역
    global od_data
    od_df = copy.deepcopy(od_data)
    od_df.columns = ['승차 구역명']
    od_list = od_data['구역명']
    for od in od_list:
        od_df['하차 구역명'] = od
        if od == od_list[0]:
            df_od = copy.deepcopy(od_df)
        else:
            df_od = df_od.append(od_df)
    df_od.reset_index(drop = True, inplace = True)
    # 일자
    date_list = pd.date_range(start, end)
    for d in date_list:
        df_od['승차 일자'] = d
        if d == date_list[0]:
            df_date = copy.deepcopy(df_od)
        else:
            df_date = df_date.append(df_od)
    df_date.reset_index(drop = True, inplace = True)
    del date_list, df_od
    # 일자 특성 구분
    global calendar_data
    df_date = df_date.merge(calendar_data, on = '승차 일자', how = 'left')
    # 승차 시각
    global time_data
    time_df = copy.deepcopy(time_data) 
    h_list = time_df['운행 시간']
    for h in h_list:
        df_date['승차 시각'] = h
        if h == h_list[0]:
            df_h = copy.deepcopy(df_date)
        else:
            df_h = df_h.append(df_date)
    df_h.reset_index(drop = True, inplace = True)
    del df_date
    # 승차 분
    m_list = [0, 30]
    for m in m_list:
        df_h['승차 분'] = m
        if m == 0:
            df_time = copy.deepcopy(df_h)
        else:
            df_time = df_time.append(df_h)
    df_time.reset_index(drop = True, inplace = True)
    del m_list, df_h
    # 집계 결과와 합치기
    table = df_time.merge(df, on = ['승차 일자', '승차 일자 특성 구분', '승차 시각', '승차 분', '승차 구역명', '하차 구역명'], how = 'left')
    table['운행 수'] = table['운행 수'].fillna(0)
    table = table[['승차 일자', '승차 일자 특성 구분', '승차 시각', '승차 분', '승차 구역명', '하차 구역명', '운행 수']]
    table.sort_values(['승차 구역명', '하차 구역명', '승차 일자', '승차 시각', '승차 분'], inplace = True)
    table.reset_index(drop = True, inplace = True)
    return(table)


# 월별 시간별 이동 내역 Heatmap 생성
def mt_heatmap(data, car_type):
    # 분석 테이블 기간 설정 (1년)
    end_date = datetime.datetime.strptime(end_ym, '%Y-%m') + relativedelta(months = 1) - relativedelta(days = 1)
    start_date = end_date - relativedelta(years = 1) + relativedelta(days = 1)
    if data['승차 일자'].min().date() > start_date.date():
        start_date = data['승차 일자'].min().date()  
    # 분석 테이블 생성
    table_result = create_table_heatmap(data, start_date, end_date)
    # 월별 시간별 집계 (mean)
    table_result['승차 연'] = table_result['승차 일자'].apply(lambda x: x.year)
    table_result['승차 월'] = table_result['승차 일자'].apply(lambda x: x.month)
    mt_table = pd.DataFrame(table_result.groupby(['승차 연', '승차 월', '승차 일자 특성 구분', '승차 시각', '승차 구역명', '하차 구역명'])['운행 수'].mean())
    mt_table.reset_index(inplace = True)
    mt_table.columns = ['승차 연', '승차 월', '승차 일자 특성 구분', '승차 시각', '승차 구역명', '하차 구역명', '평균 운행 수']
    mt_table['연월/시'] = mt_table['승차 연'].astype(str) + '년 / ' + mt_table['승차 월'].astype(str) + '월 / ' + mt_table['승차 시각'].astype(str) + '시'
    # 평일, 시간 구분
    for i in mt_table['승차 일자 특성 구분'].unique():
        heatmap_table = mt_table[mt_table['승차 일자 특성 구분'] == i]
        # heatmap 작성
        fig = px.density_heatmap(heatmap_table, y = '승차 구역명', x = '하차 구역명', z = '평균 운행 수', 
                                 animation_frame = '연월/시', histfunc = 'avg', width = 900, height = 900, 
                                 color_continuous_scale = ['#ffffff', '#f00c0c'],
                                 range_color = [0, heatmap_table['평균 운행 수'].max()],
                                 title = '월별 시간별 평균 운행수 heatmap_' + i + '_' + car_type
                                 ) 
        plot_name = 'Heatmap_월별 시간별 이동내역_' + i + '_' + car_type + '.html'
        pio.write_html(fig, heatmap_dir_num + plot_name, auto_play = False)
        print('[월별 시간별 이동내역 - ' + i + '_' + car_type + '] \n' + heatmap_dir_num + plot_name + '\n')

# 5. 승하차 분석

전체 차량

In [ ]:
mt_heatmap(data_all, '전체')

[월별 시간별 이동내역 - 평일_전체] 
C:/19Project/05.TRV/01.Pred_Demand/3. 승하차 분석_20211006_1/Heatmap_월별 시간별 이동내역_평일_전체.html

[월별 시간별 이동내역 - 휴일_전체] 
C:/19Project/05.TRV/01.Pred_Demand/3. 승하차 분석_20211006_1/Heatmap_월별 시간별 이동내역_휴일_전체.html



센터 차량

In [ ]:
mt_heatmap(data_center, '센터차량')

[월별 시간별 이동내역 - 평일_센터차량] 
C:/19Project/05.TRV/01.Pred_Demand/3. 승하차 분석_20211006_1/Heatmap_월별 시간별 이동내역_평일_센터차량.html

[월별 시간별 이동내역 - 휴일_센터차량] 
C:/19Project/05.TRV/01.Pred_Demand/3. 승하차 분석_20211006_1/Heatmap_월별 시간별 이동내역_휴일_센터차량.html



임차 택시

In [ ]:
mt_heatmap(data_taxi, '임차택시')

[월별 시간별 이동내역 - 평일_임차택시] 
C:/19Project/05.TRV/01.Pred_Demand/3. 승하차 분석_20211006_1/Heatmap_월별 시간별 이동내역_평일_임차택시.html

[월별 시간별 이동내역 - 휴일_임차택시] 
C:/19Project/05.TRV/01.Pred_Demand/3. 승하차 분석_20211006_1/Heatmap_월별 시간별 이동내역_휴일_임차택시.html

